In [1]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras
import numpy as np

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
commission = 5
buy_times = 0
min_loss = 0
max_profit = 0

C:\Users\Bikal\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

In [3]:
symbols = """
select distinct(name) from stockdataweekly order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [4]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [5]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = None
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
   
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
      

sql1= """
select distinct(name) from stockdataweekly order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)

<h1> Updating resources

In [6]:
cursor = len(symbol_list)
percent = cursor // 100


counter = 0
percentage = 0
for symbol in symbol_list:
    if(counter % percent == 0 ):
        print(percentage , "% done")
        percentage += 1
    symbol_data = """
    select * from stockdataweekly where name = '%s'
    """ % symbol
    counter += 1
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)

0 % done
1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done
100 % done

In [7]:
sql_dates = """
select distinct(date) from stockdataweekly order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Loading KNN Model

In [8]:
import pickle

pickle_in = open("KNN-ModelWeekly63.pickle","rb")
model = pickle.load(pickle_in)

<h1> Trading strategy with the training sets generation

In [9]:
def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global buy_times
    global min_loss
    global max_profit
    
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdataweekly where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    
                    
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        training_inst = training_set_extractor(data_till_date, date)
                        training_inst = np.asarray(training_inst)
                        for_predict = np.append([tf.keras.utils.normalize(training_inst[0])], [tf.keras.utils.normalize(training_inst[1])])
                        if(model.predict([for_predict]) == 1):
                            buy_stocks(current_price, current_symbol, date)  
                            
                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                win += 1
                                current_symbol.flag_for_wins = True
                        
                        
                    if current_symbol.stop_order > current_price:
                        
                        
                        current_symbol.stop_order = None
                        print("Selling ",position[symbol], "stocks of", symbol, " at price", current_price, "on date", date)
                        sold = current_price * position[symbol] - commission
                        profit_today = sold - current_symbol.bought * position[symbol]
                        
                        if profit_today > 0:                           
                            if current_symbol.flag_for_wins == False:
                                win += 1
                            current_symbol.flag_for_wins = False
                            
                        else:
                            loss += 1                           
                            current_symbol.flag_for_wins = False
                            
                        
                        profit += profit_today
                        
                        if profit < min_loss:
                            min_loss = profit
                        if profit > max_profit:
                            profit = profit_today
                            
                        
                        remaining_cash = remaining_cash + sold
                        del position[symbol]
                        current_symbol.bought = None
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("Profit",profit)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [10]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def training_set_extractor(data, date):
    past_ten_weeks = data.tail(10)
    difference = abs(len(past_ten_weeks['2. high'].tolist()) - 10)
    zeros_list = [0] * (difference)
    training_instance = [zeros_list[:], zeros_list[:], 'na']
        
    training_instance[0].extend(past_ten_weeks['2. high'].tolist()[:])
    training_instance[1].extend(past_ten_weeks['5. volume'].tolist()[:])
    return training_instance[:]   

def buy_stocks(current_price, current_symbol, date):
    global position
    global remaining_cash
    global risk
    global commission
    
    quantity = (0.02 * remaining_cash) // (current_price - (current_symbol.support * 0.97))
              
    if quantity > 1:
        current_symbol.bought = current_price
        current_symbol.stop_order = current_symbol.support * 0.97
        remaining_cash = remaining_cash - commission - (quantity * current_price)
        position[current_symbol.symbol] = quantity
        print("Buying ", quantity, "stocks of ", current_symbol.symbol, "At price", current_symbol.bought, "on date", date)
    else:
        pass
        

In [11]:
automatic_trader(symbol_list)

Buying  72.0 stocks of  ASRVP At price 23.0 on date 2000-01-28
Buying  37.0 stocks of  BKH At price 25.0 on date 2000-01-28
Buying  274.0 stocks of  CMU At price 5.875 on date 2000-01-28
Buying  155.0 stocks of  DNR At price 4.563 on date 2000-01-28
Buying  33.0 stocks of  EGN At price 18.94 on date 2000-01-28
Buying  24.0 stocks of  HLX At price 34.38 on date 2000-01-28
Buying  44.0 stocks of  ICAD At price 3.81 on date 2000-01-28
Buying  16.0 stocks of  RJF At price 20.44 on date 2000-01-28
Buying  11.0 stocks of  SMMF At price 41.0 on date 2000-01-28
Buying  6.0 stocks of  UIS At price 36.06 on date 2000-01-28
Buying  28.0 stocks of  WETF At price 4.88 on date 2000-01-28
Buying  15.0 stocks of  WTFC At price 15.94 on date 2000-01-28
Buying  59.0 stocks of  AAME At price 2.875 on date 2000-02-04
Buying  60.0 stocks of  BIF At price 8.75 on date 2000-02-04
Buying  13.0 stocks of  CNX At price 12.44 on date 2000-02-04
Buying  6.0 stocks of  CVR At price 23.25 on date 2000-02-04
Buying 

Buying  3.0 stocks of  SENEA At price 13.75 on date 2000-11-17
Buying  5.0 stocks of  AOI At price 3.75 on date 2000-11-24
Selling  6.0 stocks of CVR  at price 19.13 on date 2000-11-24
Buying  2.0 stocks of  DGSE At price 7.75 on date 2000-11-24
Buying  3.0 stocks of  EEI At price 7.375 on date 2000-11-24
Selling  3.0 stocks of NMI  at price 11.25 on date 2000-11-24
Selling  3.0 stocks of NYMX  at price 2.281 on date 2000-11-24
Buying  3.0 stocks of  SR At price 23.5 on date 2000-11-24
Buying  3.0 stocks of  CFFN At price 15.19 on date 2000-12-08
Buying  5.0 stocks of  JHS At price 14.13 on date 2000-12-08
Buying  3.0 stocks of  GIM At price 6.563 on date 2001-01-05
Buying  2.0 stocks of  NXN At price 13.69 on date 2001-01-05
Selling  2.0 stocks of AXE  at price 23.44 on date 2001-01-12
Selling  2.0 stocks of INO  at price 1.063 on date 2001-01-12
Buying  2.0 stocks of  AIRI At price 2.062 on date 2001-01-26
Buying  6.0 stocks of  PIM At price 6.73 on date 2001-02-16
Buying  4.0 stocks

Selling  4.0 stocks of CCCL  at price 8.56 on date 2008-10-10
Selling  7.0 stocks of SGOC  at price 6.7 on date 2008-10-10
Buying  11.0 stocks of  NVCN At price 0.4 on date 2008-10-24
Selling  2.0 stocks of RJN  at price 7.46 on date 2008-11-14
Buying  7.0 stocks of  CADC At price 2.75 on date 2008-11-21
Selling  9.0 stocks of ESXB  at price 3.15 on date 2008-12-12
Selling  2.0 stocks of AWF  at price 8.25 on date 2009-01-09
Selling  3.0 stocks of DNP  at price 7.9 on date 2009-01-09
Selling  2.0 stocks of MNP  at price 10.67 on date 2009-01-09
Selling  2.0 stocks of NHS  at price 7.41 on date 2009-01-09
Selling  3.0 stocks of GFF  at price 9.58 on date 2009-01-16
Selling  3.0 stocks of RJA  at price 7.43 on date 2009-04-09
Buying  2.0 stocks of  BIOC At price 4.13 on date 2009-04-17
Buying  2.0 stocks of  HBM At price 6.1765 on date 2009-04-17
Buying  2.0 stocks of  LINU At price 6.0 on date 2009-04-17
Buying  2.0 stocks of  CWB At price 31.43 on date 2009-05-08
Buying  2.0 stocks of 

Buying  2.0 stocks of  BLMT At price 12.98 on date 2013-06-07
Buying  2.0 stocks of  FSBW At price 16.2 on date 2013-06-07
Buying  2.0 stocks of  GURU At price 19.92 on date 2013-06-07
Buying  2.0 stocks of  QDEF At price 25.67 on date 2013-06-07
Selling  2.0 stocks of ELD  at price 48.59 on date 2013-07-26
Buying  3.0 stocks of  WMGIZ At price 3.37 on date 2013-07-26
Buying  2.0 stocks of  BSCM At price 20.22 on date 2013-09-20
Buying  207.0 stocks of  TPIV At price 0.0201 on date 2013-10-25
Buying  2.0 stocks of  YDIV At price 21.5 on date 2013-11-08
Selling  2.0 stocks of MARPS  at price 16.49 on date 2013-12-06
Selling  2.0 stocks of SCHP  at price 53.589 on date 2013-12-06
Buying  2.0 stocks of  USMV At price 29.57 on date 2013-12-06
Buying  2.0 stocks of  JASN At price 10.5 on date 2013-12-13
Buying  2.0 stocks of  BSCL At price 20.31 on date 2014-01-10
Buying  2.0 stocks of  EAGLU At price 10.45 on date 2014-01-24
Selling  2.0 stocks of YDIV  at price 19.76 on date 2014-01-31
Se

In [12]:
print("Remaining Cash ::" , remaining_cash)

Remaining Cash :: 54.27479999999903


In [13]:
current_profit = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    remaining_cash += position[key] * stock_equities[key].current_price
    profit += current_profit
    
    if current_profit > 0:
        win += 1
    else:
        loss += 1
    
    if profit > max_profit:
        max_profit = profit
    
    if profit < min_loss:
        min_loss = profit
    
    
print("Maximum Profit is ", max_profit)
print("Minimum Loss is ", min_loss)
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Win Percentage")            
print((win/(loss + win))*100)

Maximum Profit is  10343.219499999996
Minimum Loss is  -1416.4584
Profit 10343.139499999996
Remaining cash  19664.038299999993
Win Percentage
51.6209476309227
